<a href="https://colab.research.google.com/github/valeriachernaya/python-ai-Valeria-Chernaya/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- [`cartoons_genre_country_duration.csv`](https://github.com/componavt/python-ai-template/blob/main/data/examples/cartoons_genre_country_duration.sparql) — жанры, страны и продолжительность мультфильмов
- [`cartoons_assessment_reviews.csv`](https://github.com/componavt/python-ai-template/blob/main/data/examples/cartoons_assessment_reviews.sparql) — оценки и рецензии мультфильмов

**Что мы делаем:**
1. Клонируем репозиторий GitHub в Colab
2. Читаем CSV-файлы в pandas DataFrame
3. Очищаем и переименовываем столбцы
4. Смотрим структуру данных и делаем быструю валидацию

## 🐱 [1] Клонируем репозиторий курса в Colab

In [10]:
# 🐱 Шаг 1. Клонируем репозиторий курса в Colab

import os

repo = "python-ai-Valeria-Chernaya"  # 🔴 ИЗМЕНЕНО: имя вашего репозитория
repo_path = f"/content/{repo}"  # абсолютный путь — не зависит от cwd

if not os.path.exists(repo_path):          # всегда проверяет /content/python-ai-Valeria-Chernaya
    !git clone -q https://github.com/valeriachernaya/python-ai-Valeria-Chernaya.git  # 🔴 ИЗМЕНЕНО: URL вашего репозитория

if os.getcwd() != repo_path:               # точное сравнение, не endswith
    %cd {repo_path}

print("✅ Репозиторий готов, теперь мы работаем внутри папки", repo)

/content/python-ai-Valeria-Chernaya
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Valeria-Chernaya


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [11]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

# 🔴 ИЗМЕНЕНО: читаем ваш файл smartphones.csv вместо примеров с мультфильмами
df_smartphones = pd.read_csv("data/smartphones.csv")

print("✅ Загружено строк в df_smartphones:", len(df_smartphones))  # 🔴 ИЗМЕНЕНО: имя переменной и текст

# 🔴 ДОБАВЛЕНО: быстрая проверка структуры ваших данных
print("\n📋 Столбцы в датасете:", df_smartphones.columns.tolist())
print("\n🔍 Первые 3 строки:")
display(df_smartphones.head(3))

✅ Загружено строк в df_smartphones: 2064

📋 Столбцы в датасете: ['smartphone', 'smartphoneLabel', 'countryLabel', 'releaseDate', 'osLabel', 'manufacturerLabel']

🔍 Первые 3 строки:


,smartphone,smartphoneLabel,countryLabel,releaseDate,osLabel,manufacturerLabel
0,http://www.wikidata.org/entity/Q15299648,Nokia X,Финляндия,2014-02-24T00:00:00Z,Nokia X,Nokia
1,http://www.wikidata.org/entity/Q4673166,Acer neoTouch P400,Китайская Республика (Тайвань),NaN,Windows Mobile,Acer
2,http://www.wikidata.org/entity/Q18395321,Alcatel One Touch Idol x+,Китай,NaN,Android,NaN


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле `smartphones.csv` есть **технические столбцы** от Викиданных, которые мешают простому анализу:

- Столбец `smartphone` с URL (ссылкой на объект Wikidata) — 🔴 **удаляем**, так как в этом анализе он не требуется.
- Столбцы с суффиксом `Label` (`smartphoneLabel`, `countryLabel`, `osLabel`, `manufacturerLabel`) содержат читаемые названия.

В этом шаге мы:
- 🔴 удалим столбец `smartphone` (URL);
- 🔴 переименуем столбцы, убрав суффикс `Label`:
  - `smartphoneLabel` → `smartphone`
  - `countryLabel` → `country`
  - `osLabel` → `os`
  - `manufacturerLabel` → `manufacturer`;
- 🔴 приведём столбец `releaseDate` к типу datetime для удобной работы с датами.

При работе с датами мы используем:
- `pd.to_datetime(..., errors="coerce")` — преобразует строки в даты, некорректные значения превращает в `NaT` (Not a Time).

> ⚠️ **Важно:** столбец с URL удалён, так как в текущем анализе ссылки на Викиданные не требуются. Если в будущем понадобится доступ к оригинальным записям Wikidata, рекомендуется сохранить копию столбца перед удалением.

In [12]:
# 🧹 Шаг 2B. Очистка и переименование столбцов для smartphones.csv

# Проверяем, нужна ли очистка (есть ли столбцы с суффиксом Label)
if "smartphoneLabel" in df_smartphones.columns or "countryLabel" in df_smartphones.columns:

    # 🔴 УДАЛЯЕМ: столбец с URL (smartphone), так как он не нужен для анализа
    if "smartphone" in df_smartphones.columns:
        df_smartphones = df_smartphones.drop(columns=["smartphone"])
        print("🗑️ Удалён столбец 'smartphone' (URL Wikidata)")

    # 🔴 ПЕРЕИМЕНОВЫВАЕМ: убираем суффикс Label
    df_smartphones = df_smartphones.rename(columns={
        "smartphoneLabel": "smartphone",    # ← убираем Label
        "countryLabel": "country",
        "osLabel": "os",
        "manufacturerLabel": "manufacturer",
    })
    print("✅ Столбцы переименованы")

    # 🔴 ОБРАБАТЫВАЕМ ДАТУ: приводим releaseDate к типу datetime
    df_smartphones["releaseDate"] = pd.to_datetime(
        df_smartphones["releaseDate"], errors="coerce"
    )
    print("✅ releaseDate преобразован в datetime (некорректные значения → NaT)")

    # 🔴 БЫСТРАЯ ПРОВЕРКА: смотрим результат
    print("\n📋 Итоговые столбцы:", df_smartphones.columns.tolist())
    print("\n🔍 Первые 3 строки после очистки:")
    display(df_smartphones.head(3))

else:
    print("⏭️ Данные уже очищены, пропускаем шаг")

print("\n✅ Данные о смартфонах готовы к анализу")

🗑️ Удалён столбец 'smartphone' (URL Wikidata)
✅ Столбцы переименованы
✅ releaseDate преобразован в datetime (некорректные значения → NaT)

📋 Итоговые столбцы: ['smartphone', 'country', 'releaseDate', 'os', 'manufacturer']

🔍 Первые 3 строки после очистки:


,smartphone,country,releaseDate,os,manufacturer
0,Nokia X,Финляндия,2014-02-24 00:00:00+00:00,Nokia X,Nokia
1,Acer neoTouch P400,Китайская Республика (Тайвань),NaT,Windows Mobile,Acer
2,Alcatel One Touch Idol x+,Китай,NaT,Android,NaN



✅ Данные о смартфонах готовы к анализу


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор DataFrame `df_smartphones`:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- 🔴 дополнительно посчитаем базовую статистику по дате выпуска (`releaseDate`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код.

> ⚠️ **Важно:** в отличие от примера с мультфильмами, у нас 🔴 **один DataFrame** вместо двух, поэтому функция вызывается только один раз.

In [13]:
# 🔍 Шаг 3. Обзор данных о смартфонах

def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔴 ИЗМЕНЕНО: вызываем функцию только для одного DataFrame
show_info(df_smartphones, "Смартфоны: название, страна, дата выпуска, ОС, производитель (df_smartphones)")

# 🔴 ДОБАВЛЕНО: базовая статистика по дате выпуска
print("\n📈 Базовая статистика по releaseDate:")
print(df_smartphones["releaseDate"].describe())

# 🔴 ДОБАВЛЕНО: количество уникальных значений в категориальных столбцах
print("\n📋 Уникальные значения по столбцам:")
print(f"  • Страны (country): {df_smartphones['country'].nunique()}")
print(f"  • Операционные системы (os): {df_smartphones['os'].nunique()}")
print(f"  • Производители (manufacturer): {df_smartphones['manufacturer'].nunique()}")

# 🔴 ДОБАВЛЕНО: проверка пропущенных значений
print("\n⚠️ Пропущенные значения:")
print(df_smartphones.isnull().sum())


📊 Смартфоны: название, страна, дата выпуска, ОС, производитель (df_smartphones)
Размер: (2064, 5)
Столбцы: smartphone, country, releaseDate, os, manufacturer

Первые строки:
                  smartphone                         country  \
0                    Nokia X                       Финляндия   
1         Acer neoTouch P400  Китайская Республика (Тайвань)   
2  Alcatel One Touch Idol x+                           Китай   
3     Acer beTouch E100/E101  Китайская Республика (Тайвань)   
4          Acer beTouch E200  Китайская Республика (Тайвань)   

                releaseDate                  os manufacturer  
0 2014-02-24 00:00:00+00:00             Nokia X        Nokia  
1                       NaT      Windows Mobile         Acer  
2                       NaT             Android          NaN  
3                       NaT  Windows Mobile 6.5          NaN  
4                       NaT  Windows Mobile 6.5          NaN  

📈 Базовая статистика по releaseDate:
count                   

## 🏭 [4] Анализ производителей: количество и "долгожители"

В этом шаге мы ответим на два важных вопроса о производителях смартфонов:

### ❓ Вопрос 1: Кто выпустил больше всего смартфонов?
- Посчитаем, сколько записей приходится на каждого производителя;
- Выведем **Топ-10 брендов** по активности;
- Отдельно выделим лидера и покажем, какую долю рынка он занимает.

### ❓ Вопрос 2: Кто самый "долгоиграющий" производитель?
- Для каждого бренда найдём **первый** и **последний** год выпуска смартфона;
- Рассчитаем **"возраст" бренда** = последний год − первый год;
- Выведем Топ-10 самых "долгоживущих" производителей;
- Покажем детали по абсолютному лидеру (первый и последний релиз, список моделей).

### 🔧 Используемые методы pandas:
| Метод | Зачем нужен |
|-------|-------------|
| `value_counts()` | Быстро посчитать количество записей по производителю |
| `groupby(...).agg()` | Одновременно найти минимальную и максимальную дату для каждого бренда |
| `dt.year` | Извлечь год из даты выпуска для расчётов |
| `sort_values()` | Отсортировать результаты по нужному столбцу |
| `idxmax()` | Найти индекс строки с максимальным значением (для детального вывода) |

> ⚠️ **Важно:** расчёт "долгожительства" выполняется только для записей с корректной датой (`releaseDate` не NaT). Производители без дат будут исключены из этого анализа, но учтены в подсчёте общего количества моделей.

In [14]:
# 🏭 Шаг 4. Анализ производителей: количество и "долгожители"

import pandas as pd

print("🔍 Анализ производителей смартфонов\n")

# ─────────────────────────────────────────────────────────────
# ВОПРОС 1: Какой производитель выпустил больше всего смартфонов?
# ─────────────────────────────────────────────────────────────
print("=" * 60)
print("📊 ВОПРОС 1: Кто выпустил больше всего смартфонов?")
print("=" * 60)

# Считаем количество смартфонов по каждому производителю
manufacturer_counts = df_smartphones["manufacturer"].value_counts()

# Топ-10 производителей
top_10_manufacturers = manufacturer_counts.head(10)

print("\n🏆 Топ-10 производителей по количеству моделей:")
for rank, (manufacturer, count) in enumerate(top_10_manufacturers.items(), 1):
    percentage = count / len(df_smartphones) * 100
    print(f"{rank}. {manufacturer:25s} → {count:3d} моделей ({percentage:5.2f}%)")

# Лидер
leader = manufacturer_counts.idxmax()
leader_count = manufacturer_counts.max()
leader_share = leader_count / len(df_smartphones) * 100

print(f"\n👑 Абсолютный лидер: {leader}")
print(f"   • Выпустил {leader_count} смартфонов из {len(df_smartphones)} в датасете")
print(f"   • Доля в выборке: {leader_share:.2f}%")

# ─────────────────────────────────────────────────────────────
# ВОПРОС 2: Какой производитель самый "долгоиграющий"?
# ─────────────────────────────────────────────────────────────
print("\n" + "=" * 60)
print("⏱️  ВОПРОС 2: Кто самый «долгоиграющий» производитель?")
print("=" * 60)

# Создаём копию для работы с датами, убираем строки без даты
df_with_dates = df_smartphones.dropna(subset=["releaseDate"]).copy()

if len(df_with_dates) > 0:
    # Извлекаем год из даты
    df_with_dates["release_year"] = df_with_dates["releaseDate"].dt.year

    # Группируем по производителю и находим первый/последний год
    manufacturer_years = df_with_dates.groupby("manufacturer")["release_year"].agg(
        first_release="min",
        last_release="max",
        models_count="count"  # количество моделей с датой
    ).reset_index()

    # Рассчитываем "возраст" бренда (разница между последним и первым релизом)
    manufacturer_years["longevity_years"] = (
        manufacturer_years["last_release"] - manufacturer_years["first_release"]
    )

    # Сортируем по "долгожительству"
    longevity_ranking = manufacturer_years.sort_values(
        "longevity_years", ascending=False
    ).reset_index(drop=True)

    print("\n🕰️  Топ-10 самых «долгоиграющих» производителей:")
    print(f"{'#':<3} {'Производитель':<25s} {'Первый год':>10s} {'Последний год':>13s} {'Возраст (лет)':>14s} {'Моделей':>8s}")
    print("-" * 75)

    for idx in range(min(10, len(longevity_ranking))):
        row = longevity_ranking.iloc[idx]
        print(f"{idx+1:<3} {row['manufacturer']:<25s} {int(row['first_release']):>10d} {int(row['last_release']):>13d} {int(row['longevity_years']):>14d} {int(row['models_count']):>8d}")

    # Детали по абсолютному лидеру по долгожительству
    if len(longevity_ranking) > 0:
        top_longevity = longevity_ranking.iloc[0]
        print(f"\n👑 Абсолютный «долгожитель»: {top_longevity['manufacturer']}")
        print(f"   • Первый релиз: {int(top_longevity['first_release'])} г.")
        print(f"   • Последний релиз: {int(top_longevity['last_release'])} г.")
        print(f"   • Активность: {int(top_longevity['longevity_years'])} лет")
        print(f"   • Моделей с датой в датасете: {int(top_longevity['models_count'])}")

        # Покажем модели этого производителя (первые и последние)
        manufacturer_models = df_with_dates[
            df_with_dates["manufacturer"] == top_longevity["manufacturer"]
        ].sort_values("releaseDate")

        if len(manufacturer_models) > 0:
            print(f"\n📱 Примеры моделей {top_longevity['manufacturer']}:")
            print("   Самые ранние:")
            for _, model in manufacturer_models.head(3).iterrows():
                year = model["releaseDate"].year if pd.notna(model["releaseDate"]) else "N/A"
                print(f"   • {model['smartphone']} ({year})")
            if len(manufacturer_models) > 3:
                print("   Самые поздние:")
                for _, model in manufacturer_models.tail(3).iterrows():
                    year = model["releaseDate"].year if pd.notna(model["releaseDate"]) else "N/A"
                    print(f"   • {model['smartphone']} ({year})")

    # Бонус: статистика по пропущенным датам
    missing_dates = len(df_smartphones) - len(df_with_dates)
    if missing_dates > 0:
        print(f"\n⚠️  Примечание: {missing_dates} записей ({missing_dates/len(df_smartphones)*100:.1f}%) не имеют даты выпуска")
        print("   → Эти записи исключены из анализа «долгожительства», но учтены в подсчёте количества моделей")

else:
    print("❌ Не удалось выполнить анализ: в датасете нет записей с корректной датой выпуска")

print("\n✅ Анализ производителей завершён")

🔍 Анализ производителей смартфонов

📊 ВОПРОС 1: Кто выпустил больше всего смартфонов?

🏆 Топ-10 производителей по количеству моделей:
1. Samsung Electronics       → 335 моделей (16.23%)
2. Xiaomi                    → 320 моделей (15.50%)
3. Huawei                    → 163 моделей ( 7.90%)
4. HTC                       → 125 моделей ( 6.06%)
5. Sony Mobile Communications →  96 моделей ( 4.65%)
6. LG Electronics            →  75 моделей ( 3.63%)
7. Apple Inc.                →  69 моделей ( 3.34%)
8. Nokia                     →  58 моделей ( 2.81%)
9. Motorola Mobility         →  52 моделей ( 2.52%)
10. Samsung Group             →  43 моделей ( 2.08%)

👑 Абсолютный лидер: Samsung Electronics
   • Выпустил 335 смартфонов из 2064 в датасете
   • Доля в выборке: 16.23%

⏱️  ВОПРОС 2: Кто самый «долгоиграющий» производитель?

🕰️  Топ-10 самых «долгоиграющих» производителей:
#   Производитель             Первый год Последний год  Возраст (лет)  Моделей
------------------------------------------

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей
  - типы оценок и результатов

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨